In [67]:
import json
import pandas as pd

In [68]:
v1=json.load(open('../../mc1_vis_system/public/MC1.json'))
nodes_df=pd.DataFrame(v1['nodes'])
links_df=pd.DataFrame(v1['links'])
merged_info=pd.read_csv('../V1/merged_info.csv') #用原始的
merged_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11069 entries, 0 to 11068
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Unnamed: 0      11069 non-null  int64  
 1   link_type       11069 non-null  object 
 2   weight          11068 non-null  float64
 3   dataset_x       11069 non-null  object 
 4   source          11069 non-null  object 
 5   target          11069 non-null  object 
 6   key             11069 non-null  int64  
 7   source_type     11069 non-null  object 
 8   source_country  3003 non-null   object 
 9   target_type     11069 non-null  object 
 10  target_country  3547 non-null   object 
dtypes: float64(1), int64(2), object(8)
memory usage: 951.4+ KB


In [69]:
nodes_df['type'].fillna("unknown",inplace=True)
consider_types=["company","organization","person","unknown"]
all_nodes=nodes_df[nodes_df['type'].isin(consider_types)]['id'].unique()
all_vessels=nodes_df[nodes_df['type']=="vessel"]['id'].unique()

In [70]:
nodes_df[nodes_df['id']=="Oceanfront Oasis Inc Carriers"]

,type,dataset,country,id
3172,unknown,MC1,NaN,Oceanfront Oasis Inc Carriers


### 3:出入度之差

In [71]:
in_out_dif={}


In [72]:
len(merged_info[merged_info['source']=="Oceanfront Oasis Inc Carriers"])

0

In [73]:
for node in all_nodes:
    in_num=len(merged_info[merged_info['target']==node])
    out_num=len(merged_info[merged_info['source']==node])
    in_out_dif[node]=abs(in_num-out_num)

In [74]:
test_max=sorted(in_out_dif.items(),key=lambda x:x[1],reverse=True)
test_max[:5]

[('160', 67),
 ('435054320', 55),
 ('903311212', 50),
 ('341411', 49),
 ('Estrella de Coral Ltd. Corporation Logistics', 45)]

In [75]:
with open("./3_inOutDif.json",'w',encoding='utf-8') as f:
    f.write(json.dumps(in_out_dif))

### 0:船的占比和出入度之差

In [94]:
chain_loc={}
for node in all_nodes:
    in_neighbor=merged_info[merged_info['target']==node]['source'].unique()
    out_neighbor=merged_info[merged_info['source']==node]['target'].unique()
    neighbor=list(set(in_neighbor)|set(out_neighbor))
    ves_neighbor=list(set(neighbor)&set(all_vessels))
    ratio=len(ves_neighbor)/len(neighbor)
    chain_loc[node]=abs(ratio*10-0.01*in_out_dif[node])
chain_loc_sort=sorted(chain_loc.items(),key=lambda x:x[1],reverse=True)
chain_loc_sort[:5]

[('4063', 9.99),
 ('740523', 9.99),
 ('56126935', 9.99),
 ('â\x80\x9cconservativeâ', 9.99),
 ('Michelle Campbell', 9.99)]

In [77]:
with open("./0_ratio.json",'w',encoding='utf-8') as f:
    f.write(json.dumps(chain_loc))

### 2:fam和org

In [78]:
one_fam={}
two_fam={}
tot_fam={}
for node in all_nodes:
    one_fam[node]=len(merged_info[((merged_info['source']==node)|(merged_info['target']==node))&(merged_info['link_type']=="family_relationship")])
    in_neighbor=merged_info[merged_info['target']==node]['source'].unique()
    out_neighbor=merged_info[merged_info['source']==node]['target'].unique()
    neighbor=list(set(in_neighbor)|set(out_neighbor))
    two_fam[node]=0
    for nei in neighbor:
        nei_nei=merged_info[((merged_info['source']==nei)|(merged_info['target']==nei))&(merged_info['link_type']=="family_relationship")]
        except_self=nei_nei[(nei_nei['source']!=node)|(nei_nei['target']!=node)]
        two_fam[node]+=len(except_self)
    tot_fam[node]=one_fam[node]+two_fam[node]*0.5
tot_fam_sort=sorted(tot_fam.items(),key=lambda x:x[1],reverse=True)
tot_fam_sort[:5]

[('png xi  Line', 641.5),
 ('341411', 622.0),
 ('Ocean Fisheries Llc', 603.5),
 ('Faroe Islands Shrimp Shark', 588.5),
 ('SeaSpray Wave SRL Solutions', 579.5)]

In [79]:
one_pol={}
two_pol={}
tot_pol={}
for node in all_nodes:
    one_pol[node]=len(merged_info[((merged_info['source']==node)&(merged_info['target_type']=="political_organization"))
                                  |(merged_info['target']==node)&(merged_info['source_type']=="political_organition")])
    in_neighbor=merged_info[merged_info['target']==node]['source'].unique()
    out_neighbor=merged_info[merged_info['source']==node]['target'].unique()
    neighbor=list(set(in_neighbor)|set(out_neighbor))
    two_pol[node]=0
    for nei in neighbor:
        nei_nei=merged_info[((merged_info['source']==nei)&(merged_info['target_type']=="political_organization"))
                                  |(merged_info['target']==nei)&(merged_info['source_type']=="political_organition")]
        except_self=nei_nei[(nei_nei['source']!=node)|(nei_nei['target']!=node)]
        two_pol[node]+=len(except_self)
    tot_pol[node]=one_pol[node]*0.8+two_pol[node]*0.4
tot_pol_sort=sorted(tot_pol.items(),key=lambda x:x[1],reverse=True)
tot_pol_sort[:5]


[('Faroe Islands Shrimp Shark', 32.4),
 ('979893388', 32.0),
 ('Mar del Oeste Pic', 26.800000000000004),
 ('png xi  Line', 25.6),
 ('Derek Burke', 25.200000000000003)]

In [102]:
tot_fam["Oceanfront Oasis Inc Carriers"],tot_pol["Oceanfront Oasis Inc Carriers"]

(56.5, 1.6)

### 4:弱连通分量船的占比和平均值的差异

In [80]:
# 读弱连通分量
with open('../../../技术储备/Tarjan/weakly connected component.json','r') as f:
    wcc=json.load(f)

ori_val={}
for i in wcc:
    components=i['component']
    ves_in_comp=[i for i in components if i in all_vessels]
    ves_ratio=len(ves_in_comp)/len(components)
    for node in components:
        ori_val[node]=ves_ratio
avg_ori_val=sum(ori_val.values())/len(ori_val)
for node in ori_val:
    ori_val[node]-=avg_ori_val


In [81]:

ori_val_sort=sorted(ori_val.items(),key=lambda x:x[1],reverse=True)
ori_val_sort[-5:]

[('dbeb3e02-dc0c-4455-a16b-2d0b9cb2c65a', -0.033547257876314615),
 ('601e76fd-8517-4269-b976-2743987a49f3', -0.033547257876314615),
 ('d4291c4b-873e-4830-b44a-85227fde3ee2', -0.033547257876314615),
 ('Erica Parks', -0.033547257876314615),
 ('Amanda Clayton', -0.033547257876314615)]

### 1：和两个可疑点的距离

In [82]:
import math

In [83]:
with open('./LINE_[x,y,id].json','r') as f:
    line_res=json.load(f)
id_coord={}
for i in line_res:
    x,y,name=i
    id_coord[name]=(x,y)

In [84]:
coord_3388=id_coord['979893388']
coord_ocean=id_coord['Oceanfront Oasis Inc Carriers']
dist_sum={}
for node in all_nodes:
    node_x,node_y=id_coord[node]
    dist_3388=math.sqrt((node_x-coord_3388[0])**2+(node_y-coord_3388[1])**2)
    dist_ocean=math.sqrt((node_x-coord_ocean[0])**2+(node_y-coord_ocean[1])**2)
    dist_sum[node]=dist_3388+dist_ocean


In [85]:
dist_sum_sort=sorted(dist_sum.items(),key=lambda x:x[1],reverse=True)
dist_sum_sort[:5]

[('Isa', 223.9060349133525),
 ('Phillips', 223.83688418580005),
 ('David Romero', 223.65057601503256),
 ('10880329', 223.639008714303),
 ('Jessica Owens', 223.34218235335612)]

In [97]:
dist_sum['Oceanfront Oasis Inc Carriers']

25.09588974151673

### 6:环数

In [86]:
with open('./cycles.json','r') as f:
    cycles=json.load(f)
    

In [87]:
cycle_cnt={}
for node in cycles:
    cycle_cnt[node]=0.01*math.sqrt(cycles[node]['count'])
for node in all_nodes:
    if node not in cycle_cnt:
        cycle_cnt[node]=0


In [88]:
cycle_cnt_sort=sorted(cycle_cnt.items(),key=lambda x:x[1],reverse=True)
cycle_cnt_sort[:5]

[('png xi  Line', 1.876566012694464),
 ('Faroe Islands Shrimp Shark', 1.816700305498956),
 ('Mar del Oeste Pic', 1.7482848738120456),
 ('SeaSpray Wave SRL Solutions', 1.7036725037400822),
 ('Ocean Fisheries Llc', 1.5640012787718558)]

### 5:pagerank看供应链位置

True

In [112]:
sum_score={}
for node in all_nodes:
    sum_score[node]=ori_val[node]+chain_loc[node]+0.1*tot_fam[node]+tot_pol[node]-0.5*dist_sum[node]+cycle_cnt[node]
sum_score_sort=sorted(sum_score.items(),key=lambda x:x[1],reverse=True)
sum_score_sort

[('979893388', 74.52565081047354),
 ('Faroe Islands Shrimp Shark', 71.09149293700801),
 ('png xi  Line', 67.16562070712692),
 ('Ocean Fisheries Llc', 66.55042707556817),
 ('341411', 64.43473562971995),
 ('SeaSpray Wave SRL Solutions', 59.74468946941161),
 ('Mar del Oeste Pic', 58.90757350618663),
 ('903311212', 51.61684474264955),
 ('Sonia Fisheries Pvt', 43.41587701018936),
 ('Playa del Cielo Kga', 42.56340125106036),
 ('Spanish Shrimp  Carriers', 24.7602045021406),
 ('2262', 20.518247736195104),
 ('160', 16.2925872132071),
 ('Estrella de Coral Ltd. Corporation Logistics', 14.95058169137951),
 ('435054320', 7.872264375900151),
 ('Faurtil FÃ¡brica Urnas TietÃª', 1.1694960916397772),
 ('Matthew Fletcher', 0.35225793141339956),
 ('9550', 0.2718628478116596),
 ('dup93', -0.8889283402634494),
 ('551', -1.3186798795410652),
 ('Larry Cisneros', -1.7189725654521126),
 ('Tristan Johnson', -2.184073593905058),
 ('2751', -2.2515922769563925),
 ('Heather Holmes', -2.2844752387803897),
 ('Stacey O

In [113]:
with open('./res.txt','w') as f:
    for i in sum_score_sort:
        f.write(str(i)+'\n')

In [114]:
sum_score["Oceanfront Oasis Inc Carriers"]

-5.257852014385234

In [126]:
res_all={}
scale=sum_score_sort[0][1]-sum_score_sort[-1][1]
print(scale)
for i in sum_score_sort:
    x,y=i
    y-=(sum_score_sort[-1][1])
    new_s=(x,y)
    res_all[x]=y/scale*100

183.49397135547994


In [124]:
with open('./fin_res.json','w') as f:
    f.write(json.dumps(res_all))